## extracting from recon3d

In [3]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [4]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
glycolysis_ppp_tca_etc= load_json_cobra_model(filename=os.path.join(model_dir,"glycolysis_ppp_tca_etc_model.json"))



Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-30


In [5]:
_REQUIRED_REACTION_ATTRIBUTES = [
    "id",
    "name",
    "metabolites",
    "lower_bound",
    "upper_bound",
#     "gene_reaction_rule",
]

_REQUIRED_METABOLITE_ATTRIBUTES = ["id", "name", "charge","formula", "compartment"]
_ORDERED_OPTIONAL_METABOLITE_KEYS = [
   
    "_bound",
    "notes",
    "annotation",
]


In [6]:
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [20]:
ppp= ['G6PDH2r',
'TALA',
'PGL',
'GND',
'RPE',
'RPI',
'TKT1',
'TKT2',
'GTHOr',
'GTHPi',
'SPODM'
] 


In [21]:
# reaction_list=[]
# for rid in glycolysis_ppp_tca_etc.reactions:
#     if rid.id in ppp:
#         reaction_list.append(rid.id)
# len(reaction_list)

In [22]:
metabolite_list=[]
n_r=0
for reaction in ppp:
    rxn = R3D.reactions.get_by_id(reaction)
    n_r = n_r +1
    for met in rxn.metabolites:
        if met.id not in metabolite_list:
            metabolite_list.append(met.id)
print("The number of reactions in this model is", n_r, "and number of metabolites is", len(metabolite_list))

The number of reactions in this model is 11 and number of metabolites is 20


In [23]:
met_df=pd.DataFrame()
for met in metabolite_list:
    r3d_met= R3D.metabolites.get_by_id(met)
    m=metabolite_to_dict(r3d_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=met_df.append(df_2)

met_df=met_df.set_index('id')
met_df

,name,charge,formula,compartment
id,,,,
6pgl_c,"6-phospho-D-glucono-1,5-lactone",-2,C6H9O9P,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
h_c,H+,0,H,c
nadp_c,Nicotinamide adenine dinucleotide phosphate,-3,C21H25N7O17P3,c
nadph_c,Nicotinamide adenine dinucleotide phosphate - ...,-4,C21H26N7O17P3,c
e4p_c,D-Erythrose 4-phosphate,-2,C4H7O7P,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
g3p_c,Glyceraldehyde 3-phosphate,-2,C3H5O6P,c
s7p_c,Sedoheptulose 7-phosphate,-2,C7H13O10P,c


In [24]:
csv_met = os.path.join(data_dir,"ppp_met_df.csv")
met_df.to_csv(csv_met)

In [25]:
rxn_df=pd.DataFrame()
reactions_not_in_r3d=[]
for rxn in ppp:
    if rxn in R3D.reactions:
        r3d_rxn= R3D.reactions.get_by_id(rxn)
        r=reaction_to_dict(r3d_rxn)
    #     print(r)
        df=pd.DataFrame.from_dict(r,orient='index')
        df=df.T
        rxn_df=rxn_df.append(df)
    else:
        reactions_not_in_r3d.append(rxn)

rxn_df=rxn_df.set_index('id')
rxn_df
reactions_not_in_r3d

[]

In [26]:
csv_rxn = os.path.join(data_dir,"ppp_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [27]:
rxn_df

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,notes,annotation
id,,,,,,,,
G6PDH2r,Glucose 6-phosphate dehydrogenase,"{'6pgl_c': 1.0, 'g6p_c': -1.0, 'h_c': 1.0, 'na...",-1000.0,1000.0,2539_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['G6PDH2r']},"{'bigg.reaction': ['G6PDH2r'], 'biocyc': ['MET..."
TALA,Transaldolase,"{'e4p_c': 1.0, 'f6p_c': 1.0, 'g3p_c': -1.0, 's...",-1000.0,1000.0,6888_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['TALA']},"{'bigg.reaction': ['TALA'], 'biocyc': ['META:T..."
PGL,6-phosphogluconolactonase,"{'6pgc_c': 1.0, '6pgl_c': -1.0, 'h2o_c': -1.0,...",0.0,1000.0,25796_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['PGL']},"{'bigg.reaction': ['PGL'], 'biocyc': ['META:6P..."
GND,Phosphogluconate dehydrogenase,"{'6pgc_c': -1.0, 'co2_c': 1.0, 'nadp_c': -1.0,...",0.0,1000.0,5226_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['GND']},"{'bigg.reaction': ['GND'], 'biocyc': ['META:RX..."
RPE,Ribulose 5-phosphate 3-epimerase,"{'ru5p__D_c': -1.0, 'xu5p__D_c': 1.0}",-1000.0,1000.0,6120_AT1 or 6120_AT2,Pentose phosphate pathway,{'original_bigg_ids': ['RPE']},"{'bigg.reaction': ['RPE'], 'biocyc': ['META:RI..."
RPI,Ribose-5-phosphate isomerase,"{'r5p_c': -1.0, 'ru5p__D_c': 1.0}",-1000.0,1000.0,22934_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['RPI']},"{'bigg.reaction': ['RPI'], 'ec-code': ['5.3.1...."
TKT1,Transketolase,"{'g3p_c': 1.0, 'r5p_c': -1.0, 's7p_c': 1.0, 'x...",-1000.0,1000.0,7086_AT1 or 84076_AT1 or 8277_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['TKT1']},"{'bigg.reaction': ['TKT1'], 'ec-code': ['2.2.1..."
TKT2,Transketolase,"{'e4p_c': -1.0, 'f6p_c': 1.0, 'g3p_c': 1.0, 'x...",-1000.0,1000.0,7086_AT1 or 84076_AT1 or 8277_AT1,Pentose phosphate pathway,{'original_bigg_ids': ['TKT2']},"{'bigg.reaction': ['TKT2'], 'biocyc': ['META:2..."
GTHOr,Glutathione oxidoreductase,"{'gthox_c': -1.0, 'gthrd_c': 2.0, 'h_c': -1.0,...",0.0,1000.0,2936_AT1,Glutamate metabolism,{'original_bigg_ids': ['GTHO']},"{'bigg.reaction': ['GTHOr'], 'biocyc': ['META:..."


In [28]:
json_rxn = os.path.join(data_dir,"ppp_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)   